In [1]:
import time
import dill
import pandas as pd
import numpy as np
import math
from matplotlib import pylab as plt
from sklearn.cluster import KMeans
from ipywidgets import interact

In [2]:
import dill
outL = dill.load(open('outlierCrash.pkd','rb'))

In [3]:
recs = {}
now = time.time()
zz = 0
cc = 0

for item in outL:
    fname = 'OutlierDFs/'+str(item[0])+'_'+str(item[1])+'.pkd'
    out1 = dill.load(open(fname,'rb'))
    d = []
    p = []
    v = []
    for ii,row in out1.iterrows():
        date1 = pd.Timestamp(row['Start_Time']).time()
        d.append(date1.hour+date1.minute/60.0 + date1.second/60.0/60.0)
        p.append(row['Precipitation(in)'])
        v.append(row['Visibility(mi)'])
        
    for jj,pval in enumerate(p):
        if math.isnan(pval):
            p[jj] = 0.0
        if math.isnan(v[jj]):
            v[jj] = 0.0
        
    X1 = []
    X2 = []

    for kk in range(len(p)):
        X1.append([p[kk],d[kk]])
        X2.append([p[kk],v[kk]])
    X1 = np.array(X1)
    X2 = np.array(X2)
    
    km1 = KMeans(n_clusters=5,max_iter=20)
    km1.fit(X1)
    
    km2 = KMeans(n_clusters=5,max_iter=20)
    km2.fit(X2)
    
    centers1 = km1.cluster_centers_
    centers2 = km2.cluster_centers_
    
    rec = []
    
    if sum(centers1.T[0] > 0.03) >= 3:
        rec.append('Drainage')
    if (sum(centers1.T[1] <= 6) + sum(centers1.T[1] >= 20)) >= 2:
        rec.append('Lighting/Visibility')
    if sum(centers2.T[0] > 0.03) >= 3 and 'Drainage' not in rec:
        rec.append('Drainage')
    if sum(centers2.T[1] < 5) >= 3:
        rec.append('Visibility')
    if not rec:
        rec.append('Volume/Geometry')
        
    recs[item] = rec
    if zz % 100 == 0:
        print(zz)
        print(time.time()-now)
    zz += 1

0
0.07774686813354492


<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)


100
5.158949136734009


<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Pos

200
10.460664987564087


...

<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (5). Pos

11300
566.9689302444458


<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (5). Pos

11400
571.39981508255


<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km2.fit(X2)
<ipython-input-3-cec4cce8d91e>:37: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Pos

In [35]:
recs2 = {}

for item in recs:
    recs2[item] = ', '.join(recs[item])

In [37]:
dill.dump(recs2,open('recommendations.pkd','wb'))

In [26]:
b = 0
for item in recs:
    if recs[item] == ['Lighting/Visibility', 'Drainage', 'Visibility']:
        b += 1

In [35]:
from matplotlib import pylab as plt
from ipywidgets import interact
from sklearn.cluster import KMeans

def plot_kmeans(X, n=3, start=None):
    colors = ['#ba2121ff', '#42a5f5ff', '#efa016ff', '#000000ff', '#f231f2', '#6f7c91ff']

    if start is not None:
        n = start.shape[0]
    else:
        start = X[::((X.shape[0]//n)+1),:]
    
    def func(train_steps=0):
        
        if train_steps:
            km = KMeans(n_clusters=n, max_iter=train_steps, n_init=1)#, init=start)
            km.fit(X)
            print(km.cluster_centers_)
            centers = km.cluster_centers_.T
            cols = [colors[label] for label in km.labels_]
        else:
            centers = start.T
            cols = '0.5'
        if centers.shape[0] > 2:
            from mpl_toolkits.mplot3d import Axes3D            
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.scatter(*X.T, s=20, c=cols)
            ax.scatter(*centers, c=colors[:len(centers.T)], s=150, marker='*',linewidth=1, edgecolors='k')
        else:
            plt.scatter(*X.T, s=20, c=cols)
            plt.scatter(*centers, c=colors[:len(centers.T)], marker='*', s=150,linewidth=1, edgecolors='k')
        #plt.axis('image')
        plt.title('Precipitation vs Time')
        plt.xlabel('Precipitation (in)')
        plt.ylabel('Time (hrs)')
    
    return func

In [48]:
interact(plot_kmeans(np.array(X), n=5), train_steps=(0, 20, 1));

interactive(children=(IntSlider(value=0, description='train_steps', max=20), Output()), _dom_classes=('widget-…

In [13]:
centers = km.cluster_centers_.T

In [14]:
centers

array([[-8.67361738e-19,  2.06122449e-02,  0.00000000e+00,
         6.83333333e-03,  1.42857143e-03],
       [ 1.54832086e+01,  1.06570522e+01,  1.19315278e+01,
         1.07810046e+01,  1.11421693e+01],
       [ 8.06142857e+01,  5.16428571e+01,  1.65916667e+01,
         6.58983333e+01,  3.52071429e+01]])

In [15]:
centers.shape

(3, 5)